In [1]:
import pandas as pd
import numpy as np
import os
import sys
import fileinput




In [9]:
#Import Source .res.csv, whittle back to just .csv format

inputFileName = r"P:\projects\MichaelWarne\Models\BM_WarnePests_output.res.csv"
outputFileName = r"P:\projects\MichaelWarne\Models\BM_WarnePests_output_RemoveHeaders.csv"

haveHitDateLine = False
linesToSkipAfterDate = 1
counter = 0

with open(inputFileName, "r") as inputFile:
    with open(outputFileName, "w") as outputFile:
        
        for lineIn in inputFile:
            
            if lineIn[0:4] == 'Date':
                #the [0:4] above includes first index position, but excludes last
                haveHitDateLine = True
                #Write the column names
                outputFile.write(lineIn)
            
            if haveHitDateLine:
                
                if counter > linesToSkipAfterDate:
                    outputFile.write(lineIn)
                
                counter += 1

inputFile.close()
outputFile.close()
print("Header removal trimming done")
        

Header removal trimming done


In [12]:
#Process the data

allData = pd.read_csv(outputFileName)

#Add year, month and fin year

allData['Year'] = pd.DatetimeIndex(allData['Date']).year
allData['Month'] = pd.DatetimeIndex(allData['Date']).month
allData['FinYear'] = allData['Year']

#Set fin year

allData.loc[allData['Month'] >= 7, 'FinYear'] = allData['Year'] + 1

#allData.head()


,Date,1>138014A>Constituents@24-D@Downstream Flow Mass,2>138014A>Constituents@Ametryn@Downstream Flow Mass,3>138014A>Constituents@Atrazine@Downstream Flow Mass,4>138014A>Constituents@Chlorpyrifos@Downstream Flow Mass,5>138014A>Constituents@Diuron@Downstream Flow Mass,6>138014A>Constituents@Fipronil@Downstream Flow Mass,7>138014A>Constituents@Fluroxypyr@Downstream Flow Mass,8>138014A>Constituents@Haloxyfop@Downstream Flow Mass,9>138014A>Constituents@Hexazinone@Downstream Flow Mass,...,1443>Outlet Node9>Constituents@Tebuthiuron@Upstream Flow Mass,1444>Outlet Node9>Constituents@Terbuthylazine@Downstream Flow Mass,1445>Outlet Node9>Constituents@Terbuthylazine@Upstream Flow Mass,1446>Outlet Node9>Constituents@Triclopyr@Downstream Flow Mass,1447>Outlet Node9>Constituents@Triclopyr@Upstream Flow Mass,1448>Outlet Node9>Downstream Flow,1449>Outlet Node9>Upstream Flow,Year,Month,FinYear
0,1986-07-01,0,0.000000e+00,0.0,0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.048661,1986,7,1987
1,1986-07-02,0,0.000000e+00,0.0,0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.378046,1986,7,1987
2,1986-07-03,0,0.000000e+00,0.0,0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.193559,1986,7,1987
3,1986-07-04,0,2.159136e-11,0.0,0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.468292,1986,7,1987
4,1986-07-05,0,7.343637e-08,0.0,0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.996719,1986,7,1987


In [16]:
#Filter Cols
requiredElements = ['link for catchment SC #12', 'Outlet Node9', 'Outlet Node13', 'Outlet Node14',
                 'Outlet Node18', 'Outlet Node19', 'Outlet Node20', 'Outlet Node55']

reqdCols = ['Date', 'Year', 'Month', 'FinYear']

#Iterate over column names
for columnName in allData:
    
    needed = False
    
    for elementName in requiredElements:
        
        if(elementName in columnName):
            needed = True
            break
    
    if needed:
        reqdCols.append(columnName)

print(reqdCols)

['Date', 'Year', 'Month', 'FinYear', '116>Outlet Node13>Constituents@24-D@Downstream Flow Mass', '117>Outlet Node13>Constituents@Ametryn@Downstream Flow Mass', '118>Outlet Node13>Constituents@Atrazine@Downstream Flow Mass', '119>Outlet Node13>Constituents@Chlorpyrifos@Downstream Flow Mass', '120>Outlet Node13>Constituents@Diuron@Downstream Flow Mass', '121>Outlet Node13>Constituents@Fipronil@Downstream Flow Mass', '122>Outlet Node13>Constituents@Fluroxypyr@Downstream Flow Mass', '123>Outlet Node13>Constituents@Haloxyfop@Downstream Flow Mass', '124>Outlet Node13>Constituents@Hexazinone@Downstream Flow Mass', '125>Outlet Node13>Constituents@Imazapic@Downstream Flow Mass', '126>Outlet Node13>Constituents@Imidacloprid@Downstream Flow Mass', '127>Outlet Node13>Constituents@Isoxaflutole@Downstream Flow Mass', '128>Outlet Node13>Constituents@MCPA@Downstream Flow Mass', '129>Outlet Node13>Constituents@Metribuzin@Downstream Flow Mass', '130>Outlet Node13>Constituents@Metsulfuron-methyl@Downstre

In [18]:
trimmedCSVFile = r"P:\projects\MichaelWarne\Models\BM_WarnePests_output_Trimmed.csv"
trimmedTable = allData.to_csv(trimmedCSVFile, index=False, columns=reqdCols)
print("Saved")

Saved


In [25]:
trimmedDF = pd.read_csv(trimmedCSVFile)

#renaming attempt
colNameDict = {}
#aggColNames = []
aggColNames = {}

for columnName in trimmedDF:
    
    newName = columnName
    
    if '>' in columnName:
        pos = columnName.index('>')
        newName = columnName[pos+1:len(columnName)]#last is index, not count, and not inclusive
        #aggColNames.append(newName)
        aggColNames[newName] = 'sum'
        #colNameDict[columnName] = newName
    
    colNameDict[columnName] = newName


#print(colNameDict)
trimmedDF = trimmedDF.rename(columns=colNameDict)
#trimmedDF.columns = trimmedDF.columns.to_series().map(colNameDict)

trimmedDF.head()


,Date,Year,Month,FinYear,Outlet Node13>Constituents@24-D@Downstream Flow Mass,Outlet Node13>Constituents@Ametryn@Downstream Flow Mass,Outlet Node13>Constituents@Atrazine@Downstream Flow Mass,Outlet Node13>Constituents@Chlorpyrifos@Downstream Flow Mass,Outlet Node13>Constituents@Diuron@Downstream Flow Mass,Outlet Node13>Constituents@Fipronil@Downstream Flow Mass,...,Outlet Node9>Constituents@S-metolachlor@Downstream Flow Mass,Outlet Node9>Constituents@S-metolachlor@Upstream Flow Mass,Outlet Node9>Constituents@Tebuthiuron@Downstream Flow Mass,Outlet Node9>Constituents@Tebuthiuron@Upstream Flow Mass,Outlet Node9>Constituents@Terbuthylazine@Downstream Flow Mass,Outlet Node9>Constituents@Terbuthylazine@Upstream Flow Mass,Outlet Node9>Constituents@Triclopyr@Downstream Flow Mass,Outlet Node9>Constituents@Triclopyr@Upstream Flow Mass,Outlet Node9>Downstream Flow,Outlet Node9>Upstream Flow
0,1986-07-01,1986,7,1987,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048661
1,1986-07-02,1986,7,1987,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.378046
2,1986-07-03,1986,7,1987,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.193559
3,1986-07-04,1986,7,1987,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.468292
4,1986-07-05,1986,7,1987,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.996719


In [27]:
monthlyDF = trimmedDF.groupby(['Year', 'Month', 'FinYear']).agg(aggColNames)
monthlyDF.head()

Outlet Node13>Constituents@24-D@Downstream Flow Mass  \
Year Month FinYear                                                         
1986 7     1987                                              0.000000      
     8     1987                                              0.097467      
     9     1987                                              0.000000      
     10    1987                                              0.000000      
     11    1987                                              0.000000      

                    Outlet Node13>Constituents@Ametryn@Downstream Flow Mass  \
Year Month FinYear                                                            
1986 7     1987                                          0.000000e+00         
     8     1987                                          7.966577e-07         
     9     1987                                          0.000000e+00         
     10    1987                                          0.000000e+00         
     11    1987                                          0.000000e+00         

                    Outlet Node13>Constituents@Atrazine@Downstream Flow Mass  \
Year Month FinYear                                                             
1986 7     1987                                              0.000000          
     8     1987                                              0.023153          
     9     1987                                              0.000000          
     10    1987                                              0.000000          
     11    1987                                              0.000000          

                    Outlet Node13>Constituents@Chlorpyrifos@Downstream Flow Mass  \
Year Month FinYear                                                                 
1986 7     1987                                                     0              
     8     1987                                                     0              
     9     1987                                                     0              
     10    1987                                                     0              
     11    1987                                                     0              

                    Outlet Node13>Constituents@Diuron@Downstream Flow Mass  \
Year Month FinYear                                                           
1986 7     1987                                              0.000000        
     8     1987                                              0.008565        
     9     1987                                              0.000000        
     10    1987                                              0.000000        
     11    1987                                              0.000000        

                    Outlet Node13>Constituents@Fipronil@Downstream Flow Mass  \
Year Month FinYear                                                             
1986 7     1987                                                     0          
     8     1987                                                     0          
     9     1987                                                     0          
     10    1987                                                     0          
     11    1987                                                     0          

                    Outlet Node13>Constituents@Fluroxypyr@Downstream Flow Mass  \
Year Month FinYear                                                               
1986 7     1987                                              0.000000            
     8     1987                                              0.154269            
     9     1987                                              0.000000            
     10    1987                                              0.000000            
     11    1987                                              0.000000            

                    Outlet Node13>Constituents@Haloxyfop@Downstream Flow Mass  \
Year Month FinYear 

In [28]:
finyearDF = trimmedDF.groupby(['FinYear']).agg(aggColNames)
finyearDF.head()

,Outlet Node13>Constituents@24-D@Downstream Flow Mass,Outlet Node13>Constituents@Ametryn@Downstream Flow Mass,Outlet Node13>Constituents@Atrazine@Downstream Flow Mass,Outlet Node13>Constituents@Chlorpyrifos@Downstream Flow Mass,Outlet Node13>Constituents@Diuron@Downstream Flow Mass,Outlet Node13>Constituents@Fipronil@Downstream Flow Mass,Outlet Node13>Constituents@Fluroxypyr@Downstream Flow Mass,Outlet Node13>Constituents@Haloxyfop@Downstream Flow Mass,Outlet Node13>Constituents@Hexazinone@Downstream Flow Mass,Outlet Node13>Constituents@Imazapic@Downstream Flow Mass,...,Outlet Node9>Constituents@S-metolachlor@Downstream Flow Mass,Outlet Node9>Constituents@S-metolachlor@Upstream Flow Mass,Outlet Node9>Constituents@Tebuthiuron@Downstream Flow Mass,Outlet Node9>Constituents@Tebuthiuron@Upstream Flow Mass,Outlet Node9>Constituents@Terbuthylazine@Downstream Flow Mass,Outlet Node9>Constituents@Terbuthylazine@Upstream Flow Mass,Outlet Node9>Constituents@Triclopyr@Downstream Flow Mass,Outlet Node9>Constituents@Triclopyr@Upstream Flow Mass,Outlet Node9>Downstream Flow,Outlet Node9>Upstream Flow
FinYear,,,,,,,,,,,,,,,,,,,,,
1987,3.495899,0.001110,8.568874,0,2.145548,0,4.717811,0.079014,6.327627e-06,0.422707,...,0.355562,1.431222,0.000945,0.005225,1.424920e-06,0.000004,0.002437,0.009007,2513.692491,23295.760047
1988,0.194800,0.000206,0.169841,0,0.093579,0,0.402807,0.000000,3.999795e-09,0.020150,...,0.091141,0.765016,0.000763,0.011213,3.334604e-07,0.000003,0.000914,0.009010,1537.519443,22893.315684
1989,1.688287,0.013880,3.174364,0,1.732671,0,2.387675,0.000000,1.326152e-05,0.013768,...,4.538553,4.849017,0.582272,0.595460,2.614943e-04,0.000266,0.296318,0.303039,640173.445946,660840.300430
1990,0.022023,0.000195,0.199474,0,0.033385,0,0.060629,0.000000,1.602100e-09,0.012762,...,1.277021,3.284956,0.238968,0.252053,4.263163e-05,0.000045,0.052899,0.057932,279138.572594,299898.445693
1991,0.000763,0.000004,0.000718,0,0.002641,0,0.008817,0.000000,2.104473e-126,0.000455,...,0.063293,0.071201,0.115970,0.129805,1.091338e-05,0.000012,0.014318,0.016489,124906.091534,145615.309506


In [32]:
monthlyTotalsCSVFile = r"P:\projects\MichaelWarne\Models\BM_WarnePests_output_Trimmed_Monthly.csv"
monthlyTotals = monthlyDF.to_csv(monthlyTotalsCSVFile)
print("Saved")

Saved


In [33]:
annualTotalsCSVFile = r"P:\projects\MichaelWarne\Models\BM_WarnePests_output_Trimmed_FinYear.csv"
annualTotals = finyearDF.to_csv(annualTotalsCSVFile)
print("Saved")

Saved
